# Phase 2: Model Training and Experimentation

This notebook will focus on training and optimizing our model. Our objectives include
1. Experiment tracking with MLflow
2. Cross-validation
3. Hyperparameter optimization
4. Model evaluation and selection
5. Model versioning and registration

In [1]:
# ignore this: required for run_notebooks.sh
%pip install mlflow optuna --quiet

/Users/nic/git/AmesHousingPredictor/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import optuna
import xgboost as xgb

print("Loading data...")
import ames_notebooks
from app.data_ingestion.read_data import DataReader

reader = DataReader()
train_data, test_data = reader.load_train_test()
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

Loading data...
Train shape: (1460, 80)
Test shape: (1459, 79)


In [3]:
# get preprocessing pipeline
from app.pipelines.preprocessing import get_fitted_pipelines
feature_preprocessor, target_transformer = get_fitted_pipelines(train_data)

In [4]:
X = train_data.drop('SalePrice', axis=1)
y = train_data['SalePrice']

# apply pipelines/transformations
X_processed = feature_preprocessor.transform(X)
y_processed = target_transformer.transform(y)

# Split data into train and validation sets
# use _val to prevent confusion between the test dataset
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y_processed, 
    test_size=0.2, 
    random_state=42
)

# Apply preprocessing
print("Applying preprocessing pipeline...")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

Applying preprocessing pipeline...
X_train shape: (1168, 241)
X_val shape: (292, 241)


## Baseline Model Development

Let's start with a simple baseline model using XGBoost with default parameters. This will give us a reference point for further improvements.

In [5]:
from app.training.trainer import ModelTrainer

model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

trainer = ModelTrainer(tracking_uri="./mlruns")
baseline_model, train_metrics, val_metrics = trainer.train(model, "xgboost-baseline", X_train, y_train, X_val, y_val)

print("\nTraining Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nValidation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

[0]	validation_0-rmse:0.33887
[99]	validation_0-rmse:0.14973


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [13:48:45] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/12 13:48:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [13:48:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)
2025/11/12 13:48:48 INFO mlflow.models.model: Found the following environment variables used during model inference: [API_KEY, STRIPE_API_KEY]. Please check if you need to set them when 


Training Metrics:
train_rmse: 0.0078
train_mae: 0.0054
train_r2: 0.9996

Validation Metrics:
test_rmse: 0.1497
test_mae: 0.1038
test_r2: 0.8799


Registered model 'xgboost-baseline' already exists. Creating a new version of this model...
Created version '7' of model 'xgboost-baseline'.


## Hyperparameter Optimization with Optuna

Now that we have a baseline model, let's use Optuna to find better hyperparameters for our XGBoost model. We'll define an objective function that Optuna will optimize using cross-validation scores.

In [6]:
# objective function for Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42
    }
    
    # get model with suggested parameters
    model = xgb.XGBRegressor(**params)
    
    # cross-validation
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # mean negative RMSE (Optuna minimizes objective)
    return -cv_scores.mean()

# Create and run Optuna study
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-11-12 13:48:48,362] A new study created in memory with name: no-name-38721353-5d72-4fa0-86c9-2d7d98888e9d


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial:
  Value:  0.1232095428370713
  Params: 
    max_depth: 4
    learning_rate: 0.01819413361560552
    n_estimators: 721
    min_child_weight: 6
    subsample: 0.6304701750629337
    colsample_bytree: 0.67748179186674
    reg_alpha: 6.117201107029435e-08
    reg_lambda: 0.4848445852613158


In [7]:
best_params = study.best_params
best_params['random_state'] = 42
model = xgb.XGBRegressor(**best_params)

optimized_model, train_metrics, val_metrics = trainer.train(model, "xgboost-optimized", X_train, y_train, X_val, y_val)

print("\nTraining Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nValidation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

[0]	validation_0-rmse:0.42725
[100]	validation_0-rmse:0.18090
[200]	validation_0-rmse:0.14626
[300]	validation_0-rmse:0.13866
[400]	validation_0-rmse:0.13570
[500]	validation_0-rmse:0.13487
[600]	validation_0-rmse:0.13482
[700]	validation_0-rmse:0.13502
[720]	validation_0-rmse:0.13504


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [13:49:26] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/12 13:49:29 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [13:49:29] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)



Training Metrics:
train_rmse: 0.0652
train_mae: 0.0469
train_r2: 0.9722

Validation Metrics:
test_rmse: 0.1350
test_mae: 0.0881
test_r2: 0.9023


Registered model 'xgboost-optimized' already exists. Creating a new version of this model...
Created version '2' of model 'xgboost-optimized'.


## Model Comparison

Let's compare the performance of our baseline and optimized models to see the improvement from hyperparameter optimization.

In [8]:
from app.training.trainer import evaluate_model

# compare models on validation set
baseline_metrics = evaluate_model(baseline_model, X_val, y_val)
optimized_metrics = evaluate_model(optimized_model, X_val, y_val)

print("Baseline Model Metrics:")
for metric, value in baseline_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nOptimized Model Metrics:")
for metric, value in optimized_metrics.items():
    print(f"{metric}: {value:.4f}")

improvement = (baseline_metrics['rmse'] - optimized_metrics['rmse']) / baseline_metrics['rmse'] * 100
print(f"\nRMSE Improvement: {improvement:.2f}%")

Baseline Model Metrics:
rmse: 0.1497
mae: 0.1038
r2: 0.8799

Optimized Model Metrics:
rmse: 0.1350
mae: 0.0881
r2: 0.9023

RMSE Improvement: 9.81%


In [9]:
# Compose the preprocessor and trained model into a single sklearn-compatible object
# and use TransformedTargetRegressor so the target transformer is applied automatically.
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Build an sklearn Pipeline that first applies the feature preprocessor then the model.
# Note: feature_preprocessor and model are expected to be already fitted objects.
skl_pipeline = Pipeline([
    ("preprocessor", feature_preprocessor),
    ("regressor", model),
])

# Wrap with TransformedTargetRegressor so predictions are returned on the original scale.
full_pipeline = TransformedTargetRegressor(regressor=skl_pipeline, transformer=target_transformer)
full_pipeline.fit(X, y)

# Save the composed object (sklearn pipeline or fallback wrapper)
models_path = Path("../models")
models_path.mkdir(parents=True, exist_ok=True)
joblib.dump(full_pipeline, models_path / "full_pipeline.joblib")
print(f"Saved full pipeline to {models_path / 'full_pipeline.joblib'}")

# Example: predict on a single row from the test set (keeps original columns)
example_row = test_data.iloc[[0]]  # DataFrame with one row
pred = full_pipeline.predict(example_row)
print("Example prediction (SalePrice):", pred)


Saved full pipeline to ../models/full_pipeline.joblib
Example prediction (SalePrice): [123347.48]
